In [1]:
import eodal
import geopandas as gpd
import os

import matplotlib.pyplot as plt


from datetime import datetime
from eodal.config import get_settings
from eodal.core.scene import SceneCollection
from eodal.core.sensors.sentinel2 import Sentinel2
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs

from pathlib import Path
from typing import List

In [2]:
Settings = get_settings()
Settings.USE_STAC = True

In [3]:
def preprocess_sentinel2_scenes(
    ds: Sentinel2,
    target_resolution: int,
) -> Sentinel2:
    """
    Resample Sentinel-2 scenes and mask clouds, shadows, and snow
    based on the Scene Classification Layer (SCL).

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
        interfacing libraries!

    :param target_resolution:
        spatial target resolution to resample all bands to.
    :returns:
        resampled, cloud-masked Sentinel-2 scene.
    """
    # resample scene
    ds.resample(inplace=True, target_resolution=target_resolution)
    # mask clouds, shadows, and snow
    ds.mask_clouds_and_shadows(inplace=True)
    return ds

def get_sentinel2_scene(
    collection: str,
    time_start: datetime,
    time_end: datetime,
    geom: Path,
    cloud_pixel_percentage: int = 10
) -> SceneCollection:
    # ------------------------- Metadata Filters ---------------------------
    metadata_filters: List[Filter] = [
        Filter('cloudy_pixel_percentage','<', cloud_pixel_percentage),
        Filter('processing_level', '==', 'Level-2A')
    ]

    #%% query the scenes available (no I/O of scenes, this only fetches metadata)
    feature = Feature.from_geoseries(gpd.read_file(geom).dissolve().geometry)

    mapper_configs = MapperConfigs(
        collection=collection,
        time_start=time_start,
        time_end=time_end,
        feature=feature,
        metadata_filters=metadata_filters
    )

    mapper_configs.to_yaml('temp/sample_mapper_call.yaml')
    mapper = Mapper(mapper_configs)
    mapper.query_scenes()
    mapper.data

    scene_kwargs = {
        'scene_constructor': Sentinel2.from_safe,
        'scene_constructor_kwargs': {},
        'scene_modifier': preprocess_sentinel2_scenes,
        'scene_modifier_kwargs': {'target_resolution': 10}
    }

    mapper.load_scenes(scene_kwargs=scene_kwargs)

    return mapper.data

In [4]:
collection: str = 'sentinel2-msi'
time_start: datetime = datetime(2022,6,1)  # year, month, day (incl.)
time_end: datetime = datetime(2022,6,30)   # year, month, day (incl.)
geom: Path = '../images/bounding_box/bounding_box_eth_centrum_eschikon.geojson'

scenes = get_sentinel2_scene(collection, time_start, time_end, geom)
scenes.plot(band_selection=['red'], figsize=(60,20))

2023-03-22 11:29:51,358 eodal        INFO     Starting extraction of sentinel2 scenes


KeyError: 'B'

In [20]:
# set environment variable to enable working with STAC from Planetary Computer
!set USE_STAC=True

In [21]:
# define a function for Sentinel-2 data extraction from the archive
import numpy as np
import matplotlib as plt

from datetime import datetime
from typing import Any, Dict

from eodal.core.scene import SceneCollection
from eodal.core.sensors.sentinel2 import Sentinel2
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper,MapperConfigs

from shapely.geometry import MultiPoint, MultiPolygon, Point, Polygon

def preprocess_sentinel2_scenes(
    ds: Sentinel2,
    target_resolution: int,
) -> Sentinel2:
    """
    Resample Sentinel-2 scenes and mask clouds, shadows, and snow
    based on the Scene Classification Layer (SCL).

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
    interfacing libraries!

    :param target_resolution:
        spatial target resolution to resample all bands to.
    :returns:
        resampled, cloud-masked Sentinel-2 scene.
    """
    # resample scene
    ds.resample(inplace=True, target_resolution=target_resolution)
    # mask clouds, shadows, and snow
    ds.mask_clouds_and_shadows(inplace=True, cloud_classes=[3, 8, 9, 10, 11])
    return ds

def get_s2_data(
        time_start: datetime,
        time_end: datetime,
        scene_cloud_cover_threshold: float = 80,
        feature_cloud_cover_threshold: float = 50,
        spatial_resolution: int = 10
) -> SceneCollection:
    metadata_filters = [
        Filter('cloudy_pixel_percentage','<', scene_cloud_cover_threshold),
        Filter('processing_level', '==', 'Level-2A')
    ]

    # feature = Feature('zurich', Point([12991, 5267205]), epsg = 32633)
    feature = Feature('test', Polygon(shell=((13001, 5267205), (12991, 5267305), (12891, 5267205), (12991, 5267105))), epsg=32633)

    mapper_configs = MapperConfigs(
        collection='sentinel2-msi',
        time_start=time_start,
        time_end=time_end,
        feature=feature,
        metadata_filters=metadata_filters
    )

    mapper = Mapper(mapper_configs)

    mapper.query_scenes()

    scene_kwargs = {
        'scene_constructor': Sentinel2.from_safe,            # this tells the mapper how to read and load the data (i.e., Sentinel-2 scenes)
        'scene_constructor_kwargs': {},                      # here you could specify which bands to read
        'scene_modifier': preprocess_sentinel2_scenes,       # this tells the mapper about (optional) pre-processing of the loaded scenes (must be a callable)
        'scene_modifier_kwargs': {'target_resolution': 10}   # here, you have to specify the value of the arguments the `scene_modifier` requires
    }
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    scenes_to_del = []
    mapper.metadata['scene_used'] = 'yes'
    for scene_id, scene in mapper.data:

        # check if scene is blackfilled (nodata); if yes continue
        if scene.is_blackfilled:
            scenes_to_del.append(scene_id)
            mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [blackfill]'
            continue

        # check cloud coverage (including shadows and snow) of the field parcel
        feature_cloud_cover = scene.get_cloudy_pixel_percentage(cloud_classes=[3, 8, 9, 10, 11])

        # if the scene is too cloudy, we skip it
        if feature_cloud_cover > feature_cloud_cover_threshold:
            scenes_to_del.append(scene_id)
            mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [clouds]'
            continue

        # calculate the NDVI and MSAVI
        scene.calc_si('NDVI', inplace=True)
        scene.calc_si('MSAVI', inplace=True)

    # delete scenes too cloudy or containing only no-data
    for scene_id in scenes_to_del:
        del mapper.data[scene_id]
    
    return mapper

In [22]:
# plot parcel and its temporal development
def plot_scenes(res: SceneCollection) -> None:
    f, axes = plt.subplots(ncols=len(res), nrows=3, figsize=(26,14))
    idx = 0
    for scene_id, scene in res:
        scene.plot_multiple_bands(
            band_selection=['nir_1', 'red' ,'green'],
            ax=axes[0,idx]
        )
        axes[0,idx].set_title(scene_id)
        axes[0,idx].set_xlabel('')
        axes[0,idx].get_xaxis().set_ticks([])
         # plot the MSAVI
        scene.plot_band(
            'MSAVI',
            colormap='summer',
            colorbar_label='MSAVI [-]',
            vmin=0,
            vmax=0.8,
            ax=axes[1,idx]
        )
        # plot the MSAVI
        scene.plot_band(
            'NDVI',
            colormap='summer',
            colorbar_label='NDVI [-]',
            vmin=0,
            vmax=1.,
            ax=axes[2,idx]
        )
        # set y and x ticks as well as title strings at the outer bounds, only, of to improve readability
        if idx > 0:
            for jdx in range(3):
                axes[jdx,idx].get_yaxis().set_ticks([])
                axes[jdx,idx].set_ylabel('')
        for jdx in range(1,3):
            for tdx in range(len(res)):
                axes[jdx,tdx].set_title('')
                if jdx == 1:
                    axes[jdx,tdx].get_xaxis().set_ticks([])
                    axes[jdx,tdx].set_xlabel('')
        idx += 1


In [23]:
data = get_s2_data(datetime(2022,1,1), datetime(2022,10,1))

2023-03-20 15:09:54,584 eodal        INFO     Starting extraction of sentinel2 scenes


KeyboardInterrupt: 